![Img](https://app.theheadstarter.com/static/hs-logo-opengraph.png)

# Headstarter Codebase RAG Project

![Screenshot 2024-11-25 at 7 12 58 PM](https://github.com/user-attachments/assets/0bd67cf0-43d5-46d2-879c-a752cae4c8e3)

# Install Necessary Libraries

In [1]:
! pip install pygithub langchain langchain-community openai tiktoken pinecone-client langchain_pinecone sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.9/375.9 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.7 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successf

In [2]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from langchain_pinecone import PineconeVectorStore
from langchain.embeddings import OpenAIEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
from google.colab import userdata
from pinecone import Pinecone
import os
import tempfile
from github import Github, Repository
from git import Repo
from openai import OpenAI
from pathlib import Path
from langchain.schema import Document
from pinecone import Pinecone

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


# Clone a GitHub Repo locally

In [3]:
#Avoid having limits or Github by cloning the repo
def clone_repo(repo_url):
  repo_name = repo_url.split("/")[-1] #Split by the '/' and get the repo name
  repo_path = f"/content/{repo_name}" #set path to save in files
  Repo.clone_from(repo_url, str(repo_path))
  return str(repo_name)

In [4]:
path = clone_repo("https://github.com/CoderAgent/SecureAgent")

In [5]:
path = "/content/SecureAgent"

In [9]:
SUPPORTED_EXTENSIONS = [".py", ".js", ".tsx", ".ts", ".java", ".cpp"]

IGNORED_DIRS = ["node_modules", ".git", "dist", "__pycache__", ".next", ".vscode", "env", "venv"]

In [10]:
def get_file_content(file_path, repo_path):
  try:
    with open(file_path, "r", encoding="utf-8") as f:
      content = f.read()

      rel_path = os.path.relpath(file_path, repo_path)

      return {
          "name": rel_path,
          "content": content
          }
  except Exception as e:
      print(f"Error reading file {file_path}: {e}")
      return None

def get_main_files_content(repo_path: str):
   """
   Get content of supported code files from the local repository.


   Args:
       repo_path: Path to the local repository


   Returns:
       List of dictionaries containing file names and contents
   """
   files_content = []


   try:
       for root, _, files in os.walk(repo_path):
           # Skip if current directory is in ignored directories
           if any(ignored_dir in root for ignored_dir in IGNORED_DIRS):
               continue


           # Process each file in current directory
           for file in files:
               file_path = os.path.join(root, file)
               if os.path.splitext(file)[1] in SUPPORTED_EXTENSIONS:
                   file_content = get_file_content(file_path, repo_path)
                   if file_content:
                       files_content.append(file_content)


   except Exception as e:
       print(f"Error reading repository: {str(e)}")


   return files_content

In [11]:
file_content = get_main_files_content(path)

In [12]:
file_content

[{'name': 'src/constants.ts',
  'content': 'import { Node } from "@babel/traverse";\nimport { JavascriptParser } from "./context/language/javascript-parser";\nimport { ChatCompletionMessageParam } from "groq-sdk/resources/chat/completions";\n\nexport interface PRFile {\n  sha: string;\n  filename: string;\n  status:\n    | "added"\n    | "removed"\n    | "renamed"\n    | "changed"\n    | "modified"\n    | "copied"\n    | "unchanged";\n  additions: number;\n  deletions: number;\n  changes: number;\n  blob_url: string;\n  raw_url: string;\n  contents_url: string;\n  patch?: string;\n  previous_filename?: string;\n  patchTokenLength?: number;\n  old_contents?: string;\n  current_contents?: string;\n}\n\nexport interface BuilderResponse {\n  comment: string;\n  structuredComments: any[];\n}\n\nexport interface Builders {\n  convoBuilder: (diff: string) => ChatCompletionMessageParam[];\n  responseBuilder: (feedbacks: string[]) => Promise<BuilderResponse>;\n}\n\nexport interface PatchInfo {\

TODO: PARSE INTO CHUNKS SOMEHOW IN ORDERT TO MAKE THIS MORE EFFICIENT

# Embeddings

In [13]:
def get_huggingface_embeddings(text, model_name="sentence-transformers/all-mpnet-base-v2"):
    model = SentenceTransformer(model_name)
    return model.encode(text)

In [14]:
text = "I am hard programmer"
embeddings = get_huggingface_embeddings(text)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [15]:
embeddings

array([ 4.42312025e-02, -2.00013574e-02, -6.21441565e-02,  2.86784172e-02,
        4.45707142e-02,  4.21667695e-02, -6.11255355e-02, -5.23322076e-02,
       -6.70412555e-04,  2.53225025e-02,  3.00895125e-02,  1.77381616e-02,
        1.52016068e-02,  6.80319816e-02,  3.88990976e-02,  4.45033237e-03,
        5.64817861e-02, -3.36263031e-02, -2.76228487e-02, -2.77310740e-02,
        7.90857058e-03,  3.88930477e-02, -2.20813118e-02,  3.58681194e-04,
       -2.20383871e-02, -2.08087396e-02,  4.76767169e-03,  1.56233483e-03,
       -2.98177283e-02,  4.07689810e-02,  5.03965281e-03,  7.89383426e-03,
       -6.33121887e-03,  7.43068056e-03,  1.61921264e-06, -1.30419536e-02,
       -1.44999241e-02,  1.35682477e-02, -6.19259179e-02,  5.87376691e-02,
       -3.64881419e-02, -8.61030165e-03,  9.02074482e-03,  2.47901883e-02,
       -1.76541903e-03,  1.26396408e-02,  6.04808144e-02,  7.24483933e-03,
        3.01606692e-02,  3.65616493e-02,  2.68061105e-02, -1.10013643e-02,
        1.33742262e-02, -

# Setting up Pinecone
**1. Create an account on [Pinecone.io](https://app.pinecone.io/)**

**2. Create a new index called "codebase-rag" and set the dimensions to 768. Leave the rest of the settings as they are.**

![Screenshot 2024-11-24 at 10 58 50 PM](https://github.com/user-attachments/assets/f5fda046-4087-432a-a8c2-86e061005238)



**3. Create an API Key for Pinecone**

![Screenshot 2024-11-24 at 10 44 37 PM](https://github.com/user-attachments/assets/e7feacc6-2bd1-472a-82e5-659f65624a88)


**4. Store your Pinecone API Key within Google Colab's secrets section, and then enable access to it (see the blue checkmark)**

![Screenshot 2024-11-24 at 10 45 25 PM](https://github.com/user-attachments/assets/eaf73083-0b5f-4d17-9e0c-eab84f91b0bc)



In [16]:
# Set the PINECONE_API_KEY as an environment variable
pinecone_api_key = userdata.get("PINECONE_API_KEY")
os.environ['PINECONE_API_KEY'] = pinecone_api_key

# Initialize Pinecone
pc = Pinecone(api_key=userdata.get("PINECONE_API_KEY"),)

# Connect to your Pinecone index
pinecone_index = pc.Index("codebase-rag")

In [17]:
vectorstore = PineconeVectorStore(index_name="codebase-rag", embedding=HuggingFaceEmbeddings())

<ipython-input-17-5982ffb8f713>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  vectorstore = PineconeVectorStore(index_name="codebase-rag", embedding=HuggingFaceEmbeddings())
<ipython-input-17-5982ffb8f713>:1: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  vectorstore = PineconeVectorStore(index_name="codebase-rag", embedding=HuggingFaceEmbeddings())


In [18]:
# Insert the codebase embeddings into Pinecone

documents = []
for file in file_content:
  doc = Document(
      page_content=f"{file['name']}\n{file['content']}",
      metadata={"source": file['name']} #keep track of the actual file/path in metadata
  )
  documents.append(doc)

vectorstore = PineconeVectorStore.from_documents(
    documents=documents,
    embedding=HuggingFaceEmbeddings(),
    index_name="codebase-rag",
    namespace="https://github.com/CoderAgent/SecureAgent"
)










<ipython-input-18-84c595a3b909>:13: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embedding=HuggingFaceEmbeddings(),


- What we want for the project is to embeed each of teh functions sperapretly not just teh files as we have right now

# Perform RAG

1. Get your Groq API Key [here](https://console.groq.com/keys)

2. Paste your Groq API Key into your Google Colab secrets, and make sure to enable permissions for it

![Screenshot 2024-11-25 at 12 00 16 AM](https://github.com/user-attachments/assets/e5525d29-bca6-4dbd-892b-cc770a6b281d)


In [19]:
client = OpenAI(
    base_url="https://api.groq.com/openai/v1",
    api_key=userdata.get("GROQ_API_KEY")
)

In [20]:
query = "How is the Java Script parser build?"

In [21]:
query_embedding = get_huggingface_embeddings(query)

In [22]:
query_embedding

array([ 5.61594628e-02, -1.49348574e-02, -4.24231775e-02,  5.37048206e-02,
       -1.93722975e-02,  5.86283440e-03, -2.41684187e-02, -1.40586952e-02,
        1.25084603e-02, -2.77117286e-02,  5.50810620e-02,  3.57271060e-02,
        6.04640320e-02,  6.38339072e-02,  4.40293625e-02, -8.79057124e-03,
       -1.67578682e-02,  9.66126751e-03, -2.58082151e-02,  2.27985755e-02,
       -7.23111909e-03,  3.62035669e-02, -7.51218526e-04,  5.56989424e-02,
       -1.12015810e-02, -1.74055938e-02, -1.95315666e-02, -1.56235807e-02,
       -7.49082863e-02, -1.07848330e-03, -6.54262155e-02, -1.73029129e-03,
       -3.48075591e-02, -1.90451313e-02,  1.21833671e-06, -2.69036591e-02,
       -1.22005036e-02,  6.77219452e-03,  7.49845663e-03,  1.62193030e-02,
        4.28462256e-04,  2.07785740e-02, -2.01051291e-02,  1.65032428e-02,
        3.53765078e-02, -4.59042303e-02,  3.06974556e-02, -3.95435691e-02,
        2.80406885e-02,  9.45940847e-04,  9.87412129e-03, -4.11928110e-02,
        6.24641264e-03,  

In [23]:
top_matches = pinecone_index.query(vector=query_embedding.tolist(), top_k=5, include_metadata=True, namespace="https://github.com/CoderAgent/SecureAgent")

In [24]:
top_matches

{'matches': [{'id': 'f2c8eb95-1f53-44cb-a4d6-474f18d1ae99',
              'metadata': {'source': 'src/context/language/javascript-parser.ts',
                           'text': 'src/context/language/javascript-parser.ts\n'
                                   'import { AbstractParser, EnclosingContext '
                                   '} from "../../constants";\n'
                                   'import * as parser from "@babel/parser";\n'
                                   'import traverse, { NodePath, Node } from '
                                   '"@babel/traverse";\n'
                                   '\n'
                                   'const processNode = (\n'
                                   '  path: NodePath<Node>,\n'
                                   '  lineStart: number,\n'
                                   '  lineEnd: number,\n'
                                   '  largestSize: number,\n'
                                   '  largestEnclosingContext: Node | n

In [25]:
contexts = [item['metadata']['text'] for item in top_matches['matches']]

In [26]:
contexts

['src/context/language/javascript-parser.ts\nimport { AbstractParser, EnclosingContext } from "../../constants";\nimport * as parser from "@babel/parser";\nimport traverse, { NodePath, Node } from "@babel/traverse";\n\nconst processNode = (\n  path: NodePath<Node>,\n  lineStart: number,\n  lineEnd: number,\n  largestSize: number,\n  largestEnclosingContext: Node | null\n) => {\n  const { start, end } = path.node.loc;\n  if (start.line <= lineStart && lineEnd <= end.line) {\n    const size = end.line - start.line;\n    if (size > largestSize) {\n      largestSize = size;\n      largestEnclosingContext = path.node;\n    }\n  }\n  return { largestSize, largestEnclosingContext };\n};\n\nexport class JavascriptParser implements AbstractParser {\n  findEnclosingContext(\n    file: string,\n    lineStart: number,\n    lineEnd: number\n  ): EnclosingContext {\n    const ast = parser.parse(file, {\n      sourceType: "module",\n      plugins: ["jsx", "typescript"], // To allow JSX and TypeScript

In [27]:
augmented_query = "<CONTEXT>\n" + "\n\n-------\n\n".join(contexts[ : 10]) + "\n-------\n</CONTEXT>\n\n\n\nMY QUESTION:\n" + query

In [29]:
print(augmented_query)

<CONTEXT>
src/context/language/javascript-parser.ts
import { AbstractParser, EnclosingContext } from "../../constants";
import * as parser from "@babel/parser";
import traverse, { NodePath, Node } from "@babel/traverse";

const processNode = (
  path: NodePath<Node>,
  lineStart: number,
  lineEnd: number,
  largestSize: number,
  largestEnclosingContext: Node | null
) => {
  const { start, end } = path.node.loc;
  if (start.line <= lineStart && lineEnd <= end.line) {
    const size = end.line - start.line;
    if (size > largestSize) {
      largestSize = size;
      largestEnclosingContext = path.node;
    }
  }
  return { largestSize, largestEnclosingContext };
};

export class JavascriptParser implements AbstractParser {
  findEnclosingContext(
    file: string,
    lineStart: number,
    lineEnd: number
  ): EnclosingContext {
    const ast = parser.parse(file, {
      sourceType: "module",
      plugins: ["jsx", "typescript"], // To allow JSX and TypeScript
    });
    let larges

In [36]:
system_prompt = f"""You are a Senior Software engineer, specializing in TypeScript.

Answer any questions I have about the codebase, based on all the context provided.
Always consider all of the context provided when forming a reponse.
Lets think step by step. Verify step by step.
"""


llm_response = client.chat.completions.create(
    model="llama-3.1-70b-versatile",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": augmented_query}
    ]
)

response = llm_response.choices[0].message.content

In [37]:
print(response)

The JavaScript parser is built as a class named `JavascriptParser` which implements the `AbstractParser` interface. 

Here is how it's built:

```typescript
export class JavascriptParser implements AbstractParser {
  findEnclosingContext(
    file: string,
    lineStart: number,
    lineEnd: number
  ): EnclosingContext {
    // implementation
  }

  dryRun(file: string): { valid: boolean; error: string } {
    // implementation
  }
}
```

It has two methods:

- `findEnclosingContext`: This method finds the enclosing context of a piece of code in a JavaScript file. It takes the file contents and the start and end line numbers as input, and returns the enclosing context as an `EnclosingContext` object.

- `dryRun`: This method checks if the file is valid JavaScript. It takes the file contents as input, and returns a boolean indicating whether the file is valid and an error message if the file is not valid.

This parser is created as an instance and registered in the `EXTENSIONS_TO_PARSE